# Yves Leconte

In [120]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import plotly.express as px
from torch.utils.data import DataLoader
from torch import tensor
from datasets import Dataset
from transformers import BertTokenizer, DistilBertModel, DistilBertConfig
from tqdm.notebook import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [121]:
dataset = pd.read_csv('../data/sentiment_analysis_for_financial_news_[from_kaggle].csv', sep='^([^,]+),', engine='python',encoding='latin-1')
dataset.rename(columns={' Sentence;;;': 'Sentence'}, inplace=True)
dataset.drop(columns=['Unnamed: 0'], inplace=True)

We have an unbalanced dataset, let's derive some inverse-frequency based weights to be used later in the cross entropy loss

In [122]:
dataset['Sentiment'] = dataset['Sentiment'].replace({"\"neutral": 'neutral', "\"positive": 'positive', "\"negative": 'negative'})
dataset['Sentiment'].value_counts()

Sentiment
neutral     2879
positive    1363
negative     604
Name: count, dtype: int64

In [123]:
n_samples = 4846

In [124]:
freq_neutral = dataset['Sentiment'].value_counts()['neutral']/n_samples
freq_positive = dataset['Sentiment'].value_counts()['positive']/n_samples
freq_negative = dataset['Sentiment'].value_counts()['negative']/n_samples

class_weights = [1/freq_negative, 1/freq_positive, 1/freq_neutral]

Let's make the data readable for HuggingFace's Bert based tokenizer

In [125]:
dataset = Dataset.from_pandas(dataset)

In [126]:
dataset

Dataset({
    features: ['Sentiment', 'Sentence'],
    num_rows: 4846
})

In [127]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

dataset = dataset.shuffle(seed=2024)

def preprocessing_fn(examples):
    tokenized_batch = tokenizer(
        examples["Sentence"],
        add_special_tokens=False,
        truncation=True,
        max_length=256,
        padding='max_length',
        return_attention_mask=False
    )

    # Using one-hot encoding is very convenient for multi label classification
    labels = torch.tensor([0 if sentiment == 'negative' else 1 if sentiment == 'positive' else 2 for sentiment in examples['Sentiment']])
    one_hot_labels = F.one_hot(labels, num_classes=3).float()
    
    tokenized_batch = {k: tensor(v) for k, v in tokenized_batch.items()}
    tokenized_batch['labels'] = one_hot_labels
    return tokenized_batch


split_dataset = dataset.select(range(n_samples))

tokenized_dataset = split_dataset.map(preprocessing_fn, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'labels'])

train_valid_split = tokenized_dataset.train_test_split(test_size=0.2)

train_dataset = train_valid_split['train']
valid_dataset = train_valid_split['test']

Map:   0%|          | 0/4846 [00:00<?, ? examples/s]

In [128]:
train_dataset

Dataset({
    features: ['Sentiment', 'Sentence', 'input_ids', 'token_type_ids', 'labels'],
    num_rows: 3876
})

Using a datacollator to pad the inputs sentences is particularly necessary when using Bert

In [129]:
class DataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        features = self.tokenizer.pad(
            batch, padding="longest", max_length=256, return_tensors="pt"
        )
        return features

In [130]:
data_collator = DataCollator(tokenizer)

In [131]:
batch_size = 32

train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    valid_dataset, batch_size=batch_size, collate_fn=data_collator
)

Let's build our classifier

In [132]:
class DistilBertClassifier(nn.Module):
    
    def __init__(self, num_classes=3):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.classifier = nn.Linear(768, num_classes)
        
    def forward(self, input_ids, attention_mask):
        bert_out = self.bert(input_ids, attention_mask)
        out = bert_out["last_hidden_state"][:, 0]
        logits = self.classifier(out)
        probabilities = F.softmax(logits, dim=1)
        return probabilities


In [133]:
model = DistilBertClassifier(3)
model.to(device)

DistilBertClassifier(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

We check that everything is fine

In [134]:
batch = next(iter(train_dataloader))

input_ids = batch["input_ids"].to(device)
attention_mask = batch["attention_mask"].to(device)
output = model.forward(input_ids, attention_mask)

print(output.shape)
print(type(output))
print(output)

torch.Size([32, 3])
<class 'torch.Tensor'>
tensor([[0.3219, 0.2650, 0.4130],
        [0.3153, 0.2695, 0.4153],
        [0.3271, 0.2582, 0.4146],
        [0.3211, 0.2603, 0.4186],
        [0.3267, 0.2621, 0.4111],
        [0.3203, 0.2637, 0.4160],
        [0.3212, 0.2650, 0.4138],
        [0.3221, 0.2617, 0.4162],
        [0.3237, 0.2657, 0.4107],
        [0.3171, 0.2659, 0.4170],
        [0.3200, 0.2648, 0.4153],
        [0.3210, 0.2625, 0.4165],
        [0.3266, 0.2667, 0.4067],
        [0.3187, 0.2642, 0.4171],
        [0.3283, 0.2642, 0.4074],
        [0.3155, 0.2688, 0.4157],
        [0.3219, 0.2606, 0.4175],
        [0.3275, 0.2683, 0.4042],
        [0.3269, 0.2603, 0.4128],
        [0.3334, 0.2607, 0.4059],
        [0.3247, 0.2634, 0.4120],
        [0.3219, 0.2630, 0.4151],
        [0.3207, 0.2655, 0.4138],
        [0.3310, 0.2645, 0.4045],
        [0.3217, 0.2630, 0.4153],
        [0.3212, 0.2649, 0.4139],
        [0.3206, 0.2691, 0.4103],
        [0.3263, 0.2620, 0.4117],
     

In [135]:
def validation(model, valid_dataloader, num_class=3):
    total_size = 0
    acc_total = 0
    loss_total = 0
    class_correct = [0 for _ in range(num_class)]
    class_total = [0 for _ in range(num_class)]
    criterion = nn.CrossEntropyLoss(weight=tensor(class_weights).to(device))
    model.eval()
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            input_ids = batch["input_ids"]
            labels = batch["labels"]
            attention_mask = batch["attention_mask"]
            preds_probs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(preds_probs, labels)
            
            # Convert from one-hot to indices
            labels = torch.argmax(labels, dim=1)
            predicted_labels = torch.argmax(preds_probs, dim=1)

            acc = (predicted_labels == labels)
            total_size += acc.shape[0]
            acc_total += acc.sum().item()
            loss_total += loss.item()
            
            for i in range(num_class):
                class_total[i]+=torch.sum(labels==i).item()
                class_correct[i]+=torch.sum((labels==i) & (predicted_labels==i)).item()
            
    model.train()
    class_acc = [100*class_correct[i]/class_total[i] if class_total[i]!=0 else 0 for i in range(num_class)]
    return loss_total / len(valid_dataloader), acc_total / total_size, class_acc

In [136]:
def training(model, n_epochs, train_dataloader, valid_dataloader, lr=1e-6, num_class=3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, eps=1e-8)
    list_val_acc = []
    list_val_acc_class = []
    list_train_acc = []
    list_train_acc_class = []
    list_train_loss = []
    list_val_loss = []
    criterion = nn.CrossEntropyLoss(weight=tensor(class_weights).to(device))
    for e in range(n_epochs):
        # ========== Training ==========

        # Set model to training mode
        model.train()
        model.to(device)

        # Tracking variables
        train_loss = 0
        epoch_train_acc = 0
        class_correct = [0 for _ in range(num_class)]
        class_total = [0 for _ in range(num_class)]
        for batch in tqdm(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            input_ids = batch["input_ids"]
            labels = batch["labels"]
            attention_mask = batch["attention_mask"]
            optimizer.zero_grad()
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # Backward pass
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().cpu().item()
            
            labels = torch.argmax(labels, dim=1)
            predicted_labels = torch.argmax(outputs, dim=1)
            acc = (predicted_labels == labels)
            epoch_train_acc += acc.float().mean().item()
            
            for i in range(num_class):
                class_total[i]+=torch.sum((labels==i)).item()
                class_correct[i]+=((labels==i) & (predicted_labels==i)).float().mean().item()
            
        list_train_acc.append(100 * epoch_train_acc / len(train_dataloader))
        list_train_acc_class.append([100*class_correct[i]/class_total[i] if class_total[i]!=0 else 0 for i in range(num_class)])
        list_train_loss.append(train_loss / len(train_dataloader))

        # ========== Validation ==========

        validation_loss, validation_accuracy, validation_accuracy_class = validation(model, valid_dataloader)
        list_val_loss.append(validation_loss)
        list_val_acc.append(validation_accuracy * 100)
        list_val_acc_class.append(validation_accuracy_class)
        print(f"Epoch {e}, Train loss: {list_train_loss[-1]}, Train acc: {list_train_acc[-1]}, Val loss: {validation_loss}, Val acc: {validation_accuracy * 100}, Val class acc: {validation_accuracy_class}")
    return list_train_loss, list_train_acc, list_train_acc_class, list_val_loss, list_val_acc, list_val_acc_class

In [137]:
list_train_loss, list_train_acc, list_train_acc_class, list_val_loss, list_val_acc, list_val_acc_class = training(model, 25, train_dataloader, valid_dataloader)

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch 0, Train loss: 3.295780714069327, Train acc: 47.84836065573771, Val loss: 3.3170356725430783, Val acc: 51.85567010309279, Val class acc: [44.62809917355372, 0.6993006993006993, 79.39609236234459]


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch 23, Train loss: 2.1322994821592443, Train acc: 81.9672131147541, Val loss: 2.3057914451338006, Val acc: 74.63917525773196, Val class acc: [88.4297520661157, 77.62237762237763, 70.15985790408526]


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch 24, Train loss: 2.096142611820231, Train acc: 82.99180327868852, Val loss: 2.2950587141212035, Val acc: 76.5979381443299, Val class acc: [89.25619834710744, 73.42657342657343, 75.48845470692717]


In [138]:
train_losses = pd.Series(name='Train loss', data=list_train_loss)
validation_losses = pd.Series(name='Validation loss', data=list_val_loss)
train_valid_losses = pd.concat([train_losses, validation_losses], axis=1)

validation_accuracy_class = pd.DataFrame(columns=['negative','positive','neutral'], data=list_val_acc_class)


In [139]:
fig = px.line(train_valid_losses, labels={'value': 'loss', 'index': 'epoch'}, title='Train vs Validation loss')
fig

In [141]:
fig = px.line(validation_accuracy_class, labels={'value': 'accuracy', 'index': 'epoch'}, title='Accuracy per class')
fig